# problem 001

In [1]:
from warnings import filterwarnings
filterwarnings("ignore")

import pandas as pd
A = pd.read_csv("F:/DS_ML_AI_Course/START/22_23122021_PROJECT_02/training_set.csv")
A.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849.0,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,NaN,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000.0,0.0,141.0,360.0,1.0,Urban,Y


# missing data treatment

In [2]:
A.isna().sum()

Loan_ID               0
Gender               15
Married               3
Dependents           15
Education             1
Self_Employed        32
ApplicantIncome       2
CoapplicantIncome     1
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [3]:
for i in A.columns:
    if A[i].dtypes == "object":
        x = A[i].mode()[0]
        A[i] = A[i].fillna(x)
    else:
        x = A[i].mean()
        A[i] = A[i].fillna(x)

In [4]:
A.isna().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

# assigning X and Y

In [5]:
Y = A[["Loan_Status"]]
X = A.drop(labels = ["Loan_Status", "Loan_ID"], axis = 1)

# separating cat and con

In [6]:
cat = []
con = []

for i in X.columns:
    if X[i].dtypes == "object":
        cat.append(i)
    else:
        con.append(i)

In [7]:
cat

['Gender',
 'Married',
 'Dependents',
 'Education',
 'Self_Employed',
 'Property_Area']

In [8]:
con

['ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Loan_Amount_Term',
 'Credit_History']

# standardization of cat and con

In [9]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
le = LabelEncoder()
Xcat = X[cat].apply(le.fit_transform)
Xcat.head()

,Gender,Married,Dependents,Education,Self_Employed,Property_Area
0,1,0,0,0,0,2
1,1,1,1,0,0,0
2,1,1,0,0,1,2
3,1,1,0,1,0,2
4,1,0,0,0,0,2


In [10]:
ss = StandardScaler()
Xcon = pd.DataFrame(ss.fit_transform(X[con]), columns = con)
Xcon.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
0,0.072651,-0.554368,0.000000,0.279851,0.45164
1,0.000000,-0.038610,-0.219273,0.279851,0.45164
2,-0.394096,-0.554368,-0.957641,0.279851,0.45164
3,-0.462412,0.252103,-0.314547,0.279851,0.45164
4,0.097389,-0.554368,-0.064454,0.279851,0.45164


In [11]:
X = Xcon.join(Xcat)
X.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender,Married,Dependents,Education,Self_Employed,Property_Area
0,0.072651,-0.554368,0.000000,0.279851,0.45164,1,0,0,0,0,2
1,0.000000,-0.038610,-0.219273,0.279851,0.45164,1,1,1,0,0,0
2,-0.394096,-0.554368,-0.957641,0.279851,0.45164,1,1,0,0,1,2
3,-0.462412,0.252103,-0.314547,0.279851,0.45164,1,1,0,1,0,2
4,0.097389,-0.554368,-0.064454,0.279851,0.45164,1,0,0,0,0,2


# spiltting data into training and testing

In [12]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size= 0.2, random_state= 21)

# simple tree

In [13]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=21)

In [14]:
model = dtc.fit(xtrain, ytrain)
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

In [15]:
from sklearn.metrics import accuracy_score
tr_err = accuracy_score(ytrain, pred_tr)
ts_err = accuracy_score(ytest, pred_ts)

print("Training Error: ", tr_err)
print("Testing Error: ", ts_err)

Training Error:  1.0
Testing Error:  0.7154471544715447


# pruned tree with max depth

In [16]:
from sklearn.model_selection import GridSearchCV
tg = {"max_depth":range(2, 25, 1)}
cv = GridSearchCV(dtc, tg, scoring="accuracy", cv = 3)
cvmodel = cv.fit(xtrain, ytrain)
cvmodel.best_params_

{'max_depth': 4}

In [17]:
dtc = DecisionTreeClassifier(random_state=21, max_depth=4)

model = dtc.fit(xtrain, ytrain)
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

from sklearn.metrics import accuracy_score
tr_err = accuracy_score(ytrain, pred_tr)
ts_err = accuracy_score(ytest, pred_ts)

print("Training Error: ", tr_err)
print("Testing Error: ", ts_err)

Training Error:  0.8268839103869654
Testing Error:  0.7804878048780488


# pruned tree with min samples leaf

In [18]:
dtc = DecisionTreeClassifier(random_state=21)
from sklearn.model_selection import GridSearchCV
tg = {"min_samples_leaf":range(2, 25, 1)}
cv = GridSearchCV(dtc, tg, scoring="accuracy", cv = 3)
cvmodel = cv.fit(xtrain, ytrain)
cvmodel.best_params_

{'min_samples_leaf': 21}

In [19]:
dtc = DecisionTreeClassifier(random_state=21, min_samples_leaf=21)

model = dtc.fit(xtrain, ytrain)
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

from sklearn.metrics import accuracy_score
tr_err = accuracy_score(ytrain, pred_tr)
ts_err = accuracy_score(ytest, pred_ts)

print("Training Error: ", tr_err)
print("Testing Error: ", ts_err)

Training Error:  0.8105906313645621
Testing Error:  0.7723577235772358


# pruned tree with min samples used for splitting

In [20]:
dtc = DecisionTreeClassifier(random_state=21)
from sklearn.model_selection import GridSearchCV
tg = {"min_samples_split":range(2, 25, 1)}
cv = GridSearchCV(dtc, tg, scoring="accuracy", cv = 3)
cvmodel = cv.fit(xtrain, ytrain)
cvmodel.best_params_

{'min_samples_split': 23}

In [21]:
dtc = DecisionTreeClassifier(random_state=21, min_samples_split=23)

model = dtc.fit(xtrain, ytrain)
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

from sklearn.metrics import accuracy_score
tr_err = accuracy_score(ytrain, pred_tr)
ts_err = accuracy_score(ytest, pred_ts)

print("Training Error: ", tr_err)
print("Testing Error: ", ts_err)

Training Error:  0.8553971486761711
Testing Error:  0.7398373983739838


# random forest

In [22]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=21)

model = rfc.fit(xtrain, ytrain)
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

from sklearn.metrics import accuracy_score
tr_err = accuracy_score(ytrain, pred_tr)
ts_err = accuracy_score(ytest, pred_ts)

print("Training Error: ", tr_err)
print("Testing Error: ", ts_err)

Training Error:  1.0
Testing Error:  0.7886178861788617


# random forest with max depth

In [23]:
from sklearn.model_selection import GridSearchCV
tg = {"max_depth":range(2, 25, 1)}
cv = GridSearchCV(rfc, tg, scoring="accuracy", cv = 3)
cvmodel = cv.fit(xtrain, ytrain)
cvmodel.best_params_

{'max_depth': 2}

In [24]:
rfc = RandomForestClassifier(random_state=21, max_depth=2)

model = rfc.fit(xtrain, ytrain)
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

from sklearn.metrics import accuracy_score
tr_err = accuracy_score(ytrain, pred_tr)
ts_err = accuracy_score(ytest, pred_ts)

print("Training Error: ", tr_err)
print("Testing Error: ", ts_err)

Training Error:  0.8065173116089613
Testing Error:  0.8211382113821138


# random forest with min samples leaf

In [25]:
rfc = RandomForestClassifier(random_state=21)
from sklearn.model_selection import GridSearchCV
tg = {"min_samples_leaf":range(2, 25, 1)}
cv = GridSearchCV(rfc, tg, scoring="accuracy", cv = 3)
cvmodel = cv.fit(xtrain, ytrain)
cvmodel.best_params_

{'min_samples_leaf': 5}

In [26]:
rfc = RandomForestClassifier(random_state=21, min_samples_leaf=5)

model = rfc.fit(xtrain, ytrain)
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

from sklearn.metrics import accuracy_score
tr_err = accuracy_score(ytrain, pred_tr)
ts_err = accuracy_score(ytest, pred_ts)

print("Training Error: ", tr_err)
print("Testing Error: ", ts_err)

Training Error:  0.8187372708757638
Testing Error:  0.8048780487804879


# random forest with min samples used for splitting

In [27]:
rfc = RandomForestClassifier(random_state=21)
from sklearn.model_selection import GridSearchCV
tg = {"min_samples_split":range(2, 25, 1)}
cv = GridSearchCV(rfc, tg, scoring="accuracy", cv = 3)
cvmodel = cv.fit(xtrain, ytrain)
cvmodel.best_params_

{'min_samples_split': 20}

In [28]:
rfc = RandomForestClassifier(random_state=21, min_samples_split=20)

model = rfc.fit(xtrain, ytrain)
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

from sklearn.metrics import accuracy_score
tr_err = accuracy_score(ytrain, pred_tr)
ts_err = accuracy_score(ytest, pred_ts)

print("Training Error: ", tr_err)
print("Testing Error: ", ts_err)

Training Error:  0.8411405295315683
Testing Error:  0.8211382113821138


# Adaboost

In [29]:
from sklearn.ensemble import AdaBoostClassifier
dtc = DecisionTreeClassifier(random_state=21, max_depth=4)

abc = AdaBoostClassifier(dtc, random_state=21)
tg = {"n_estimators":range(2, 25, 1)}
cv = GridSearchCV(abc, tg, scoring= "accuracy", cv = 3)
cvmodel = cv.fit(xtrain, ytrain)
cvmodel.best_params_

{'n_estimators': 2}

In [30]:
abc = AdaBoostClassifier(dtc, random_state=21, n_estimators= 2)

model = abc.fit(xtrain, ytrain)
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

from sklearn.metrics import accuracy_score
tr_err = accuracy_score(ytrain, pred_tr)
ts_err = accuracy_score(ytest, pred_ts)

print("Training Error: ", tr_err)
print("Testing Error: ", ts_err)

Training Error:  0.8391038696537678
Testing Error:  0.6991869918699187


# KNN

In [31]:
from sklearn.neighbors import KNeighborsClassifier
knc = KNeighborsClassifier()

tg = {"n_neighbors":range(2, 25, 1)}
cv = GridSearchCV(knc, tg, scoring= "accuracy", cv = 3)
cvmodel = cv.fit(xtrain, ytrain)
cvmodel.best_params_

{'n_neighbors': 13}

In [32]:
knc = KNeighborsClassifier(n_neighbors= 13)

model = knc.fit(xtrain, ytrain)
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

from sklearn.metrics import accuracy_score
tr_err = accuracy_score(ytrain, pred_tr)
ts_err = accuracy_score(ytest, pred_ts)

print("Training Error: ", tr_err)
print("Testing Error: ", ts_err)

Training Error:  0.8065173116089613
Testing Error:  0.8048780487804879


# fetching testing set

In [33]:
B = pd.read_csv("F:/DS_ML_AI_Course/START/22_23122021_PROJECT_02/testing_set.csv")
B.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


# preprocessing

In [34]:
B.isna().sum()

Loan_ID               0
Gender               11
Married               0
Dependents           10
Education             0
Self_Employed        23
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      6
Credit_History       29
Property_Area         0
dtype: int64

In [35]:
for i in B.columns:
    if B[i].dtypes == "object":
        x = B[i].mode()[0]
        B[i] = B[i].fillna(x)
    else:
        x = B[i].mean()
        B[i] = B[i].fillna(x)

In [36]:
B.isna().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64

In [37]:
B = B.drop(labels= ["Loan_ID"], axis = 1)

In [38]:
Bcat = B[cat].apply(le.fit_transform)
Bcat.head()

,Gender,Married,Dependents,Education,Self_Employed,Property_Area
0,1,1,0,0,0,2
1,1,1,1,0,0,2
2,1,1,2,0,0,2
3,1,1,2,0,0,2
4,1,0,0,1,0,2


In [39]:
Bcon = pd.DataFrame(ss.fit_transform(B[con]), columns = con)
Bcon.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
0,0.186461,-0.673335,-0.429368,0.270603,0.47918
1,-0.352692,-0.029848,-0.166482,0.270603,0.47918
2,0.039641,0.098849,1.180807,0.270603,0.47918
3,-0.502774,0.418877,-0.593671,0.270603,0.00000
4,-0.311909,-0.673335,-0.955139,0.270603,0.47918


In [40]:
Bnew = Bcon.join(Bcat)
Bnew.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender,Married,Dependents,Education,Self_Employed,Property_Area
0,0.186461,-0.673335,-0.429368,0.270603,0.47918,1,1,0,0,0,2
1,-0.352692,-0.029848,-0.166482,0.270603,0.47918,1,1,1,0,0,2
2,0.039641,0.098849,1.180807,0.270603,0.47918,1,1,2,0,0,2
3,-0.502774,0.418877,-0.593671,0.270603,0.00000,1,1,2,0,0,2
4,-0.311909,-0.673335,-0.955139,0.270603,0.47918,1,0,0,1,0,2


# results - random forest with min samples split

In [41]:
rfc = RandomForestClassifier(random_state=21, min_samples_split=20)

model = rfc.fit(xtrain, ytrain)
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

from sklearn.metrics import accuracy_score
tr_err = accuracy_score(ytrain, pred_tr)
ts_err = accuracy_score(ytest, pred_ts)

print("Training Error: ", tr_err)
print("Testing Error: ", ts_err)

Training Error:  0.8411405295315683
Testing Error:  0.8211382113821138


In [42]:
predicted_loan_status = model.predict(Bnew)

In [43]:
temp_test_set = pd.read_csv("F:/DS_ML_AI_Course/START/22_23122021_PROJECT_02/testing_set.csv")
loan_id = temp_test_set[["Loan_ID"]]
loan_id["predicted_loan_status"] = predicted_loan_status
loan_id.head(20)

,Loan_ID,predicted_loan_status
0,LP001015,Y
1,LP001022,Y
2,LP001031,Y
3,LP001035,Y
4,LP001051,Y
5,LP001054,Y
6,LP001055,Y
7,LP001056,N
8,LP001059,Y
9,LP001067,Y


# problem 002

# preparing training set

In [44]:
A2 = A[A.Loan_Status == "Y"]

In [45]:
A

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849.00000,0.0,146.412162,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,5405.54085,1508.0,128.000000,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000.00000,0.0,66.000000,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583.00000,2358.0,120.000000,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000.00000,0.0,141.000000,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900.00000,0.0,71.000000,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106.00000,0.0,40.000000,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072.00000,240.0,253.000000,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583.00000,0.0,187.000000,360.0,1.0,Urban,Y


In [46]:
B["predicted_loan_status"] = predicted_loan_status

In [47]:
B.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,predicted_loan_status
0,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.000000,Urban,Y
1,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.000000,Urban,Y
2,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.000000,Urban,Y
3,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,0.825444,Urban,Y
4,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.000000,Urban,Y


In [48]:
B2 = B[B.predicted_loan_status == "Y"]

In [49]:
A2 = A2.drop(labels=["Loan_ID"], axis = 1)

In [50]:
A2.columns = B.columns

In [51]:
training_test = pd.concat([A2, B2])

In [52]:
training_test.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,predicted_loan_status
0,Male,No,0,Graduate,No,5849.0,0.0,146.412162,360.0,1.0,Urban,Y
2,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.000000,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.000000,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000.0,0.0,141.000000,360.0,1.0,Urban,Y
5,Male,Yes,2,Graduate,Yes,5417.0,4196.0,267.000000,360.0,1.0,Urban,Y


In [55]:
training_test.index = range(0, 726, 1)

In [56]:
training_set = training_test.rename(columns={"predicted_loan_status": "Loan_Status"})

In [57]:
training_set.shape

(726, 12)

In [58]:
training_set

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849.0,0.0,146.412162,360.0,1.000000,Urban,Y
1,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.000000,360.0,1.000000,Urban,Y
2,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.000000,360.0,1.000000,Urban,Y
3,Male,No,0,Graduate,No,6000.0,0.0,141.000000,360.0,1.000000,Urban,Y
4,Male,Yes,2,Graduate,Yes,5417.0,4196.0,267.000000,360.0,1.000000,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...
721,Male,Yes,3+,Not Graduate,Yes,4009.0,1777.0,113.000000,360.0,1.000000,Urban,Y
722,Male,Yes,0,Graduate,No,4158.0,709.0,115.000000,360.0,1.000000,Urban,Y
723,Male,No,0,Graduate,No,3250.0,1993.0,126.000000,360.0,0.825444,Semiurban,Y
724,Male,Yes,0,Graduate,No,5000.0,2393.0,158.000000,360.0,1.000000,Rural,Y


# missing data treatment

In [59]:
training_set.isna().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

# skew handling

In [60]:
training_set.skew()

ApplicantIncome      6.601206
CoapplicantIncome    2.918836
LoanAmount           2.936411
Loan_Amount_Term    -2.726971
Credit_History      -7.445761
dtype: float64

In [61]:
for col in training_set[con]:
    m = []
    for i in training_set[col]:
        if (i == 0):
            m.append(0.1)
        else:
            m.append(i)
    training_set[col] = m

In [62]:
con.remove("LoanAmount")

In [63]:
training_set[con]

,ApplicantIncome,CoapplicantIncome,Loan_Amount_Term,Credit_History
0,5849.0,0.1,360.0,1.000000
1,3000.0,0.1,360.0,1.000000
2,2583.0,2358.0,360.0,1.000000
3,6000.0,0.1,360.0,1.000000
4,5417.0,4196.0,360.0,1.000000
...,...,...,...,...
721,4009.0,1777.0,360.0,1.000000
722,4158.0,709.0,360.0,1.000000
723,3250.0,1993.0,360.0,0.825444
724,5000.0,2393.0,360.0,1.000000


In [64]:
import numpy as np
X1 = training_set[con].apply(np.log)
X1

,ApplicantIncome,CoapplicantIncome,Loan_Amount_Term,Credit_History
0,8.674026,-2.302585,5.886104,0.000000
1,8.006368,-2.302585,5.886104,0.000000
2,7.856707,7.765569,5.886104,0.000000
3,8.699515,-2.302585,5.886104,0.000000
4,8.597297,8.341887,5.886104,0.000000
...,...,...,...,...
721,8.296297,7.482682,5.886104,0.000000
722,8.332789,6.563856,5.886104,0.000000
723,8.086410,7.597396,5.886104,-0.191834
724,8.517193,7.780303,5.886104,0.000000


In [65]:
X1.skew()

ApplicantIncome     -4.115208
CoapplicantIncome   -0.266214
Loan_Amount_Term    -5.307268
Credit_History      -9.310839
dtype: float64

In [66]:
X1.shape

(726, 4)

In [67]:
X1 = training_set[cat].join(X1)

In [68]:
X1.shape

(726, 10)

# assigning X and Y

In [69]:
Y1 = training_set[["LoanAmount"]]

In [70]:
X1.head()

,Gender,Married,Dependents,Education,Self_Employed,Property_Area,ApplicantIncome,CoapplicantIncome,Loan_Amount_Term,Credit_History
0,Male,No,0,Graduate,No,Urban,8.674026,-2.302585,5.886104,0.0
1,Male,Yes,0,Graduate,Yes,Urban,8.006368,-2.302585,5.886104,0.0
2,Male,Yes,0,Not Graduate,No,Urban,7.856707,7.765569,5.886104,0.0
3,Male,No,0,Graduate,No,Urban,8.699515,-2.302585,5.886104,0.0
4,Male,Yes,2,Graduate,Yes,Urban,8.597297,8.341887,5.886104,0.0


In [71]:
X1.shape

(726, 10)

# standerdization

In [72]:
tr1 = X1[cat].apply(le.fit_transform)
tr1.head()

,Gender,Married,Dependents,Education,Self_Employed,Property_Area
0,1,0,0,0,0,2
1,1,1,0,0,1,2
2,1,1,0,1,0,2
3,1,0,0,0,0,2
4,1,1,2,0,1,2


In [73]:
tr2 = pd.DataFrame(ss.fit_transform(X1[con]), columns= con)
tr2.head()

,ApplicantIncome,CoapplicantIncome,Loan_Amount_Term,Credit_History
0,0.500047,-1.152409,0.273656,0.168373
1,-0.440216,-1.152409,0.273656,0.168373
2,-0.650983,0.880153,0.273656,0.168373
3,0.535943,-1.152409,0.273656,0.168373
4,0.391990,0.996501,0.273656,0.168373


In [74]:
trd = tr1.join(tr2)
trd.head()

,Gender,Married,Dependents,Education,Self_Employed,Property_Area,ApplicantIncome,CoapplicantIncome,Loan_Amount_Term,Credit_History
0,1,0,0,0,0,2,0.500047,-1.152409,0.273656,0.168373
1,1,1,0,0,1,2,-0.440216,-1.152409,0.273656,0.168373
2,1,1,0,1,0,2,-0.650983,0.880153,0.273656,0.168373
3,1,0,0,0,0,2,0.535943,-1.152409,0.273656,0.168373
4,1,1,2,0,1,2,0.391990,0.996501,0.273656,0.168373


In [75]:
X1 = trd

# splitting of data

In [76]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X1, Y1, test_size= 0.2, random_state= 21)

# linear regression model

In [77]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

model = lr.fit(xtrain, ytrain)

pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

In [78]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

tr_err = mean_absolute_error(ytrain, pred_tr)
ts_err = mean_absolute_error(ytest, pred_ts)

In [79]:
tr_err

37.57010883388761

In [80]:
ts_err

42.47534978070624

# Lasso

In [81]:
from sklearn.linear_model import Lasso
ls = Lasso()

tp = []
x = 0.91

for i in range(0, 50, 1):
    x = x + 0.001
    x = round(x, 3)
    tp.append(x)
    
tg = {"alpha": tp}

cv = GridSearchCV(ls, tg, scoring="neg_mean_absolute_error", cv = 3)
cvmodel = cv.fit(xtrain, ytrain)
cvmodel.best_params_

{'alpha': 0.96}

In [82]:
ls = Lasso(alpha=0.96)

model = ls.fit(xtrain, ytrain)

pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

tr_err = mean_absolute_error(ytrain, pred_tr)
ts_err = mean_absolute_error(ytest, pred_ts)

In [83]:
tr_err

37.64841481733696

In [84]:
ts_err

41.55072500901062

# Ridge

In [85]:
from sklearn.linear_model import Ridge
rr = Ridge()

tp = []
x = 0.91

for i in range(0, 50, 1):
    x = x + 0.001
    x = round(x, 3)
    tp.append(x)
    
tg = {"alpha": tp}

cv = GridSearchCV(rr, tg, scoring="neg_mean_absolute_error", cv = 3)
cvmodel = cv.fit(xtrain, ytrain)
cvmodel.best_params_

{'alpha': 0.96}

In [86]:
rr = Ridge(alpha=0.96)

model = rr.fit(xtrain, ytrain)

pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

tr_err = mean_absolute_error(ytrain, pred_tr)
ts_err = mean_absolute_error(ytest, pred_ts)

In [87]:
tr_err

37.56433281442455

In [88]:
ts_err

42.456425320705314

# preparing testing set

In [89]:
A2 = A[A.Loan_Status == "N"]

In [90]:
A2.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,LP001003,Male,Yes,1,Graduate,No,5405.54085,1508.0,128.0,360.0,1.0,Rural,N
7,LP001014,Male,Yes,3+,Graduate,No,3036.00000,2504.0,158.0,360.0,0.0,Semiurban,N
9,LP001020,Male,Yes,1,Graduate,No,12841.00000,10968.0,349.0,360.0,1.0,Semiurban,N
13,LP001029,Male,No,0,Graduate,No,1853.00000,2840.0,114.0,360.0,1.0,Rural,N
17,LP001036,Female,No,0,Graduate,No,3510.00000,0.0,76.0,360.0,0.0,Urban,N


In [93]:
B2 = B[B.predicted_loan_status == "N"]
B2.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,predicted_loan_status
7,Male,Yes,2,Not Graduate,No,3881,0,147.0,360.0,0.0,Rural,N
13,Male,Yes,2,Graduate,No,12173,0,166.0,360.0,0.0,Semiurban,N
25,Male,No,0,Graduate,No,0,24000,148.0,360.0,0.0,Rural,N
35,Male,No,0,Graduate,No,3150,0,176.0,360.0,0.0,Semiurban,N
55,Male,No,0,Graduate,No,2750,0,130.0,360.0,0.0,Urban,N


In [98]:
A2 = A2.drop(labels= ["Loan_ID"], axis = 1)

In [99]:
B2.columns = A2.columns

In [100]:
testing_set = pd.concat([A2, B2])
testing_set.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,Male,Yes,1,Graduate,No,5405.54085,1508.0,128.0,360.0,1.0,Rural,N
7,Male,Yes,3+,Graduate,No,3036.00000,2504.0,158.0,360.0,0.0,Semiurban,N
9,Male,Yes,1,Graduate,No,12841.00000,10968.0,349.0,360.0,1.0,Semiurban,N
13,Male,No,0,Graduate,No,1853.00000,2840.0,114.0,360.0,1.0,Rural,N
17,Female,No,0,Graduate,No,3510.00000,0.0,76.0,360.0,0.0,Urban,N


In [101]:
testing_set.shape

(255, 12)

In [102]:
testing_set.index = range(0, 255, 1)

In [104]:
testing_set.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,Yes,1,Graduate,No,5405.54085,1508.0,128.0,360.0,1.0,Rural,N
1,Male,Yes,3+,Graduate,No,3036.00000,2504.0,158.0,360.0,0.0,Semiurban,N
2,Male,Yes,1,Graduate,No,12841.00000,10968.0,349.0,360.0,1.0,Semiurban,N
3,Male,No,0,Graduate,No,1853.00000,2840.0,114.0,360.0,1.0,Rural,N
4,Female,No,0,Graduate,No,3510.00000,0.0,76.0,360.0,0.0,Urban,N


# missing data treatment

In [105]:
testing_set.isna().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

# skew handling

In [106]:
testing_set.skew()

ApplicantIncome      7.992601
CoapplicantIncome    6.023393
LoanAmount           2.297645
Loan_Amount_Term    -2.058502
Credit_History       0.227265
dtype: float64

In [108]:
for col in testing_set[con]:
    m = []
    for i in testing_set[col]:
        if (i == 0):
            m.append(0.1)
        else:
            m.append(i)
    testing_set[col] = m

In [109]:
testing_set[con]

,ApplicantIncome,CoapplicantIncome,Loan_Amount_Term,Credit_History
0,5405.54085,1508.0,360.0,1.0
1,3036.00000,2504.0,360.0,0.1
2,12841.00000,10968.0,360.0,1.0
3,1853.00000,2840.0,360.0,1.0
4,3510.00000,0.1,360.0,0.1
...,...,...,...,...
250,1700.00000,2900.0,360.0,0.1
251,2875.00000,2416.0,6.0,0.1
252,4333.00000,2333.0,360.0,0.1
253,3391.00000,1966.0,360.0,0.1


In [110]:
X1 = testing_set[con].apply(np.log)
X1.head()

,ApplicantIncome,CoapplicantIncome,Loan_Amount_Term,Credit_History
0,8.595180,7.318540,5.886104,0.000000
1,8.018296,7.825645,5.886104,-2.302585
2,9.460398,9.302737,5.886104,0.000000
3,7.524561,7.951559,5.886104,0.000000
4,8.163371,-2.302585,5.886104,-2.302585


In [111]:
X1.skew()

ApplicantIncome     -5.500553
CoapplicantIncome   -0.096961
Loan_Amount_Term    -6.252578
Credit_History       0.217236
dtype: float64

In [112]:
X1 = testing_set[cat].join(X1)
X1.head()

,Gender,Married,Dependents,Education,Self_Employed,Property_Area,ApplicantIncome,CoapplicantIncome,Loan_Amount_Term,Credit_History
0,Male,Yes,1,Graduate,No,Rural,8.595180,7.318540,5.886104,0.000000
1,Male,Yes,3+,Graduate,No,Semiurban,8.018296,7.825645,5.886104,-2.302585
2,Male,Yes,1,Graduate,No,Semiurban,9.460398,9.302737,5.886104,0.000000
3,Male,No,0,Graduate,No,Rural,7.524561,7.951559,5.886104,0.000000
4,Female,No,0,Graduate,No,Urban,8.163371,-2.302585,5.886104,-2.302585


# standaerdization

In [115]:
ts1 = X1[cat].apply(le.fit_transform)
ts1.head()

,Gender,Married,Dependents,Education,Self_Employed,Property_Area
0,1,1,1,0,0,0
1,1,1,3,0,0,1
2,1,1,1,0,0,1
3,1,0,0,0,0,0
4,0,0,0,0,0,2


In [118]:
ts2 = pd.DataFrame(ss.fit_transform(X1[con]), columns= con)
ts2.head()

,ApplicantIncome,CoapplicantIncome,Loan_Amount_Term,Credit_History
0,0.363749,0.829515,0.211941,1.128804
1,-0.251936,0.929241,0.211941,-0.898708
2,1.287162,1.219724,0.211941,1.128804
3,-0.778879,0.954004,0.211941,1.128804
4,-0.097103,-1.062562,0.211941,-0.898708


In [119]:
tsd = ts1.join(ts2)
tsd.head()

,Gender,Married,Dependents,Education,Self_Employed,Property_Area,ApplicantIncome,CoapplicantIncome,Loan_Amount_Term,Credit_History
0,1,1,1,0,0,0,0.363749,0.829515,0.211941,1.128804
1,1,1,3,0,0,1,-0.251936,0.929241,0.211941,-0.898708
2,1,1,1,0,0,1,1.287162,1.219724,0.211941,1.128804
3,1,0,0,0,0,0,-0.778879,0.954004,0.211941,1.128804
4,0,0,0,0,0,2,-0.097103,-1.062562,0.211941,-0.898708


# result - Lasso

In [120]:
ls = Lasso(alpha=0.96)

model = ls.fit(xtrain, ytrain)

pred_ts = model.predict(tsd)

In [121]:
predicted_loan_amount = pd.DataFrame(pred_ts, columns=["predicted_loan_amount"])

In [123]:
la = testing_set[["LoanAmount"]]

In [124]:
result2 = predicted_loan_amount.join(la)

In [125]:
result2.head(20)

,predicted_loan_amount,LoanAmount
0,164.858820,128.0
1,153.321394,158.0
2,197.699899,349.0
3,113.687098,114.0
4,115.778901,76.0
5,132.241535,133.0
6,148.310310,104.0
7,131.725143,116.0
8,146.034702,112.0
9,153.734595,151.0


# 003

# preparing training set

In [126]:
training_set.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849.0,0.1,146.412162,360.0,1.0,Urban,Y
1,Male,Yes,0,Graduate,Yes,3000.0,0.1,66.000000,360.0,1.0,Urban,Y
2,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.000000,360.0,1.0,Urban,Y
3,Male,No,0,Graduate,No,6000.0,0.1,141.000000,360.0,1.0,Urban,Y
4,Male,Yes,2,Graduate,Yes,5417.0,4196.0,267.000000,360.0,1.0,Urban,Y


In [128]:
tr_new = training_set[training_set["Loan_Amount_Term"] <= 240]

In [130]:
tr_new.shape

(70, 12)

# missing data treatment

In [132]:
tr_new.isna().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

# separating cat and con

In [136]:
cat = []
con = []

for i in tr_new.columns:
    if tr_new[i].dtypes == "object":
        cat.append(i)
    else:
        con.append(i)

In [137]:
cat

['Gender',
 'Married',
 'Dependents',
 'Education',
 'Self_Employed',
 'Property_Area',
 'Loan_Status']

In [138]:
con

['ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Loan_Amount_Term',
 'Credit_History']

In [139]:
con.remove("Loan_Amount_Term")

In [146]:
tr_new.index = range(0, 70, 1)

# assigning X and Y

In [147]:
Y3 = tr_new[["Loan_Amount_Term"]]
X3 = tr_new.drop(labels= ["Loan_Amount_Term"], axis = 1)

In [148]:
X3.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Credit_History,Property_Area,Loan_Status
0,Male,Yes,2,Graduate,No,1299.0,1086.0,17.0,1.000000,Urban,Y
1,Male,No,1,Not Graduate,No,3596.0,0.1,100.0,0.842199,Urban,Y
2,Male,Yes,3+,Not Graduate,Yes,7100.0,0.1,125.0,1.000000,Urban,Y
3,Male,Yes,1,Graduate,No,3988.0,0.1,50.0,1.000000,Urban,Y
4,Male,Yes,2,Graduate,No,6250.0,5654.0,188.0,1.000000,Semiurban,Y


# skew handling

In [149]:
X3.skew()

ApplicantIncome      5.297458
CoapplicantIncome    1.625054
LoanAmount           2.975267
Credit_History      -5.171563
dtype: float64

In [150]:
for col in X3[con]:
    m = []
    for i in X3[col]:
        if (i == 0):
            m.append(0.1)
        else:
            m.append(i)
    X3[col] = m

In [151]:
X3.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Credit_History,Property_Area,Loan_Status
0,Male,Yes,2,Graduate,No,1299.0,1086.0,17.0,1.000000,Urban,Y
1,Male,No,1,Not Graduate,No,3596.0,0.1,100.0,0.842199,Urban,Y
2,Male,Yes,3+,Not Graduate,Yes,7100.0,0.1,125.0,1.000000,Urban,Y
3,Male,Yes,1,Graduate,No,3988.0,0.1,50.0,1.000000,Urban,Y
4,Male,Yes,2,Graduate,No,6250.0,5654.0,188.0,1.000000,Semiurban,Y


In [152]:
tr1 = X3[con].apply(np.log)
tr1.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Credit_History
0,7.169350,6.990257,2.833213,0.000000
1,8.187577,-2.302585,4.605170,-0.171739
2,8.867850,-2.302585,4.828314,0.000000
3,8.291045,-2.302585,3.912023,0.000000
4,8.740337,8.640119,5.236442,0.000000


In [153]:
X3 = X3[cat].join(tr1)
X3.head()

,Gender,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status,ApplicantIncome,CoapplicantIncome,LoanAmount,Credit_History
0,Male,Yes,2,Graduate,No,Urban,Y,7.169350,6.990257,2.833213,0.000000
1,Male,No,1,Not Graduate,No,Urban,Y,8.187577,-2.302585,4.605170,-0.171739
2,Male,Yes,3+,Not Graduate,Yes,Urban,Y,8.867850,-2.302585,4.828314,0.000000
3,Male,Yes,1,Graduate,No,Urban,Y,8.291045,-2.302585,3.912023,0.000000
4,Male,Yes,2,Graduate,No,Semiurban,Y,8.740337,8.640119,5.236442,0.000000


# standerdization

In [155]:
tr1 = X3[cat].apply(le.fit_transform)
tr1.head()

,Gender,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status
0,1,1,2,0,0,2,0
1,1,0,1,1,0,2,0
2,1,1,3,1,1,2,0
3,1,1,1,0,0,2,0
4,1,1,2,0,0,1,0


In [156]:
tr2 = pd.DataFrame(ss.fit_transform(X3[con]), columns= con)
tr2.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Credit_History
0,-1.832381,0.934591,-2.695956,0.204653
1,-0.324770,-0.914007,-0.009630,-0.242343
2,0.682458,-0.914007,0.328661,0.204653
3,-0.171573,-0.914007,-1.060456,0.204653
4,0.493659,1.262794,0.947392,0.204653


In [157]:
trd = tr1.join(tr2)

In [158]:
trd.head()

,Gender,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status,ApplicantIncome,CoapplicantIncome,LoanAmount,Credit_History
0,1,1,2,0,0,2,0,-1.832381,0.934591,-2.695956,0.204653
1,1,0,1,1,0,2,0,-0.324770,-0.914007,-0.009630,-0.242343
2,1,1,3,1,1,2,0,0.682458,-0.914007,0.328661,0.204653
3,1,1,1,0,0,2,0,-0.171573,-0.914007,-1.060456,0.204653
4,1,1,2,0,0,1,0,0.493659,1.262794,0.947392,0.204653


In [159]:
X3 = trd

# splitting of data

In [160]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X3, Y3, test_size= 0.2, random_state= 21)

# linear regression

In [161]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

model = lr.fit(xtrain, ytrain)

pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

In [162]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

tr_err = mean_absolute_error(ytrain, pred_tr)
ts_err = mean_absolute_error(ytest, pred_ts)

In [163]:
tr_err

38.132717994179075

In [164]:
ts_err

43.37679019479166

# Lasso

In [165]:
from sklearn.linear_model import Lasso
ls = Lasso()

tg = {"alpha": tp}

from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(ls, tg, scoring="neg_mean_absolute_error", cv = 3)
cvmodel = cv.fit(xtrain, ytrain)
cvmodel.best_params_

{'alpha': 0.96}

In [166]:
ls = Lasso(alpha= 0.96)

model = ls.fit(xtrain, ytrain)

pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

tr_err = mean_absolute_error(ytrain, pred_tr)
ts_err = mean_absolute_error(ytest, pred_ts)

In [167]:
tr_err

38.770494125057425

In [168]:
ts_err

39.93882949016005

# Ridge

In [169]:
from sklearn.linear_model import Ridge
rr = Ridge()

tg = {"alpha": tp}

from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(rr, tg, scoring="neg_mean_absolute_error", cv = 3)
cvmodel = cv.fit(xtrain, ytrain)
cvmodel.best_params_

{'alpha': 0.96}

In [170]:
rr = Ridge(alpha= 0.96)

model = rr.fit(xtrain, ytrain)

pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

tr_err = mean_absolute_error(ytrain, pred_tr)
ts_err = mean_absolute_error(ytest, pred_ts)

In [171]:
tr_err

38.1365774192743

In [172]:
ts_err

42.22020072820457

# results - Lasso

# preparation of testing set

In [173]:
ts_new = testing_set[testing_set["Loan_Amount_Term"] <= 240]

In [175]:
ts_new.shape

(24, 12)

In [176]:
ts_new.index = range(0, 24, 1)

In [177]:
ts_new.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,Yes,0,Not Graduate,Yes,2609.0,3449.0,165.000000,180.0,0.1,Rural,N
1,Male,No,0,Not Graduate,No,3200.0,2254.0,126.000000,180.0,0.1,Urban,N
2,Male,Yes,1,Graduate,Yes,10513.0,3850.0,160.000000,180.0,0.1,Urban,N
3,Male,Yes,3+,Not Graduate,No,3522.0,0.1,81.000000,180.0,1.0,Rural,N
4,Male,Yes,3+,Not Graduate,No,3992.0,0.1,146.412162,180.0,1.0,Urban,N


In [178]:
ts_new = ts_new.drop(labels=["Loan_Amount_Term"], axis = 1)

# missing data treatment

In [179]:
ts_new.isna().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

# skew handling

In [180]:
ts_new.skew()

ApplicantIncome      2.088832
CoapplicantIncome    4.524325
LoanAmount           1.869214
Credit_History       0.192567
dtype: float64

In [184]:
for col in ts_new[con]:
    m = []
    for i in ts_new[col]:
        if (i == 0):
            m.append(0.1)
        else:
            m.append(i)
    ts_new[col] = m

In [185]:
temp = ts_new[con].apply(np.log)
temp.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Credit_History
0,7.866722,8.145840,5.105945,-2.302585
1,8.070906,7.720462,4.836282,-2.302585
2,9.260368,8.255828,5.075174,-2.302585
3,8.166784,-2.302585,4.394449,0.000000
4,8.292048,-2.302585,4.986426,0.000000


In [186]:
ts_new = ts_new[cat].join(temp)
ts_new.head()

,Gender,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status,ApplicantIncome,CoapplicantIncome,LoanAmount,Credit_History
0,Male,Yes,0,Not Graduate,Yes,Rural,N,7.866722,8.145840,5.105945,-2.302585
1,Male,No,0,Not Graduate,No,Urban,N,8.070906,7.720462,4.836282,-2.302585
2,Male,Yes,1,Graduate,Yes,Urban,N,9.260368,8.255828,5.075174,-2.302585
3,Male,Yes,3+,Not Graduate,No,Rural,N,8.166784,-2.302585,4.394449,0.000000
4,Male,Yes,3+,Not Graduate,No,Urban,N,8.292048,-2.302585,4.986426,0.000000


# standardization

In [187]:
ts1 = ts_new[cat].apply(le.fit_transform)
ts1.head()

,Gender,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status
0,1,1,0,1,1,0,0
1,1,0,0,1,0,2,0
2,1,1,1,0,1,2,0
3,1,1,3,1,0,0,0
4,1,1,3,1,0,2,0


In [188]:
ts2 = pd.DataFrame(ss.fit_transform(ts_new[con]), columns= con)
ts2.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Credit_History
0,-0.398432,0.668603,0.660696,-0.919471
1,-0.028804,0.578940,0.000862,-0.919471
2,2.124446,0.691786,0.585402,-0.919471
3,0.144762,-1.533759,-1.080249,1.100343
4,0.371523,-1.533759,0.368246,1.100343


In [189]:
tsd = ts1.join(ts2)
tsd.head()

,Gender,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status,ApplicantIncome,CoapplicantIncome,LoanAmount,Credit_History
0,1,1,0,1,1,0,0,-0.398432,0.668603,0.660696,-0.919471
1,1,0,0,1,0,2,0,-0.028804,0.578940,0.000862,-0.919471
2,1,1,1,0,1,2,0,2.124446,0.691786,0.585402,-0.919471
3,1,1,3,1,0,0,0,0.144762,-1.533759,-1.080249,1.100343
4,1,1,3,1,0,2,0,0.371523,-1.533759,0.368246,1.100343


# result

In [190]:
ls = Lasso(alpha= 0.96)

model = ls.fit(xtrain, ytrain)

pred_ts = model.predict(tsd)

In [191]:
predicted_loan_amount_term = pd.DataFrame(pred_ts, columns= ["predicted_loan_amount_term"])

In [192]:
predicted_loan_amount_term

,predicted_loan_amount_term
0,146.794819
1,148.392535
2,158.196589
3,190.956082
4,163.676818
5,145.854521
6,149.966325
7,161.467990
8,153.925928
9,176.884185


In [ ]:
la = testing_set[["Loan_Am"]]